In [3]:
# !pip install -r requirements.txt

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from pythonping import ping


import sys
import os
import json
import random
import time

import shap
import yfinance as yf
import requests
import pytest

print('Python Version:', sys.version)
print('pd', pd.__version__)
print('np', np.__version__)
print('sns', sns.__version__)
print('json', json.__version__)
print('shap', shap.__version__)
print('yfinance', yf.__version__)
print('requests', requests.__version__)
print('pytest', pytest.__version__)
# print('pythonping', pythonping.__version__)


Python Version: 3.11.3 (v3.11.3:f3909b8bc8, Apr  4 2023, 20:12:10) [Clang 13.0.0 (clang-1300.0.29.30)]
pd 2.0.1
np 1.24.3
sns 0.12.2
json 2.0.9
shap 0.46.0
yfinance 0.2.18
requests 2.30.0
pytest 8.2.2


In [107]:
import random

def get_random_decimal():
  """Generates a random decimal number between -1 and 1 (inclusive)."""
  return random.uniform(-1, 1)

# This line is optional if you want to test the function directly in the script
result = get_random_decimal()
print(result)

-0.13496552581809196


In [ ]:
# terminal command: python app/app.py --debug

In [48]:
url = 'http://127.0.0.1:5000/random_decimal'
response = requests.get(url)
print(response)
data = response.json()  # Parse the JSON response

random_number = data['random_decimal']
print(random_number)


<Response [200]>
40.79912206312349


In [49]:
url = 'http://127.0.0.1:5000/multiply_and_add'
data = {'x': -2.5, 'y': -3.1, 'z': random.uniform(-1, 1)}  # Dictionary with arguments
headers = {'Content-type': 'application/json'}  # Set content type header

response = requests.post(url, json=data, headers=headers)
print(response)
response.raise_for_status()  # Raise exception for non-200 status codes

data = response.json()
result = data['result']
print(result)

<Response [200]>
7.143362368218115


In [50]:
!python latency_test.py

Average latency: 0.92 ms
Minimum latency: 0.69 ms
Maximum latency: 8.13 ms


In [37]:
# Specify your API endpoint URL
api_url = 'http://127.0.0.1:5000/'

# Convert URL to hostname (assuming port is standard for your setup)
hostname = api_url.split("//")[1].split(":")[0]  # Extract hostname from URL
print(hostname)


try:
  # Create a ping object with desired options (adjust timeout as needed)
  pinger = ping(hostname, timeout=1000)  # 1 second timeout in milliseconds
  print(pinger)

  # Send pings and get response statistics
  response = pinger.run()
  print(response)

  # Print latency information
  if response.successful():
    print(f"Average round-trip time to {api_url}: {response.rtt_avg:.2f} ms")
    print(f"Minimum round-trip time: {response.rtt_min:.2f} ms")
    print(f"Maximum round-trip time: {response.rtt_max:.2f} ms")
    print(f"Packet loss: {response.packet_loss:.2f}%")
  else:
    print(f"Ping failed to {hostname}")

except Exception as e:
  print(f"Error occurred during ping: {e}")

127.0.0.1
Error occurred during ping: [Errno 1] Operation not permitted


In [56]:
api_url = 'http://127.0.0.1:5000/random_decimal'
# api_url = 'http://127.0.0.1:5000/multiply_and_add'


num_requests = 100

# Initialize empty list to store latencies
latencies = []

for _ in range(num_requests):
  start_time = time.time()
  response = requests.get(api_url)
  response.raise_for_status()  # Raise exception for non-2xx status codes
  end_time = time.time()

  latency = (end_time - start_time) * 1000  # Convert to milliseconds
  latencies.append(latency)

# Calculate and print summary statistics (optional)
average_latency = sum(latencies) / len(latencies)
print(f"Average latency: {average_latency:.2f} ms")
print(f"Minimum latency: {min(latencies):.2f} ms")
print(f"Maximum latency: {max(latencies):.2f} ms")

Average latency: 1.01 ms
Minimum latency: 0.73 ms
Maximum latency: 3.03 ms


# YFinance Data

In [122]:
msft = yf.Ticker("MSFT")

# get all stock info
print(msft.actions)

                           Dividends  Stock Splits
Date                                              
1987-09-21 00:00:00-04:00       0.00           2.0
1990-04-16 00:00:00-04:00       0.00           2.0
1991-06-27 00:00:00-04:00       0.00           1.5
1992-06-15 00:00:00-04:00       0.00           1.5
1994-05-23 00:00:00-04:00       0.00           2.0
...                              ...           ...
2023-05-17 00:00:00-04:00       0.68           0.0
2023-08-16 00:00:00-04:00       0.68           0.0
2023-11-15 00:00:00-05:00       0.75           0.0
2024-02-14 00:00:00-05:00       0.75           0.0
2024-05-15 00:00:00-04:00       0.75           0.0

[91 rows x 2 columns]


In [126]:
msft.major_holders

,Breakdown,Unnamed: 1
0,0.05%,% of Shares Held by All Insider
1,73.61%,% of Shares Held by Institutions
2,73.65%,% of Float Held by Institutions
3,6797,Number of Institutions Holding Shares


In [132]:
len(msft.news)

8

In [118]:
# get historical market data
hist = msft.history(period="1mo")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-06-04 00:00:00-04:00,412.429993,416.440002,409.679993,416.070007,14348900,0.0,0.0
2024-06-05 00:00:00-04:00,417.809998,424.079987,416.299988,424.010010,16988000,0.0,0.0
2024-06-06 00:00:00-04:00,424.010010,425.309998,420.579987,424.519989,14861300,0.0,0.0
2024-06-07 00:00:00-04:00,426.200012,426.279999,423.000000,423.850006,13621700,0.0,0.0
2024-06-10 00:00:00-04:00,424.700012,428.079987,423.890015,427.869995,14003000,0.0,0.0
2024-06-11 00:00:00-04:00,425.480011,432.820007,425.250000,432.679993,14551100,0.0,0.0
2024-06-12 00:00:00-04:00,435.320007,443.399994,433.250000,441.059998,22366200,0.0,0.0
2024-06-13 00:00:00-04:00,440.850006,443.390015,439.369995,441.579987,15960600,0.0,0.0
2024-06-14 00:00:00-04:00,438.279999,443.140015,436.720001,442.570007,13582000,0.0,0.0


In [133]:
yf.Ticker("VTI").actions

,Dividends,Stock Splits,Capital Gains
Date,,,
2001-06-25 00:00:00-04:00,0.1400,0.0,0.0
2001-09-24 00:00:00-04:00,0.1575,0.0,0.0
2001-12-24 00:00:00-05:00,0.1965,0.0,0.0
2002-03-25 00:00:00-05:00,0.1500,0.0,0.0
2002-06-24 00:00:00-04:00,0.1300,0.0,0.0
...,...,...,...
2023-06-23 00:00:00-04:00,0.8270,0.0,0.0
2023-09-21 00:00:00-04:00,0.7980,0.0,0.0
2023-12-21 00:00:00-05:00,1.0020,0.0,0.0


In [134]:
yf.Ticker("FXAIX").actions

,Dividends,Stock Splits,Capital Gains
Date,,,
2011-10-07 00:00:00-04:00,0.223,0.0,0.000
2011-12-16 00:00:00-05:00,0.256,0.0,0.000
2012-04-04 00:00:00-04:00,0.186,0.0,0.000
2012-07-06 00:00:00-04:00,0.288,0.0,0.000
2012-10-05 00:00:00-04:00,0.282,0.0,0.000
2012-12-14 00:00:00-05:00,0.345,0.0,0.000
2013-04-05 00:00:00-04:00,0.271,0.0,0.000
2013-07-05 00:00:00-04:00,0.279,0.0,0.000
2013-10-04 00:00:00-04:00,0.309,0.0,0.000


In [135]:
yf.Ticker("SOXQ").actions

,Dividends,Stock Splits,Capital Gains
Date,,,
2021-09-20 00:00:00-04:00,0.079,0.0,0.0
2021-12-20 00:00:00-05:00,0.148,0.0,0.0
2022-03-21 00:00:00-04:00,0.065,0.0,0.0
2022-06-21 00:00:00-04:00,0.070,0.0,0.0
2022-09-19 00:00:00-04:00,0.069,0.0,0.0
2022-12-19 00:00:00-05:00,0.067,0.0,0.0
2023-03-20 00:00:00-04:00,0.077,0.0,0.0
2023-06-20 00:00:00-04:00,0.070,0.0,0.0
2023-09-18 00:00:00-04:00,0.064,0.0,0.0


In [136]:
yf.Ticker("NVD").actions

,Dividends,Stock Splits,Capital Gains
Date,,,
2023-12-27 00:00:00-05:00,3.005,0.0,0.0
